This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. a

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections
import math

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url_test = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?\
       &start_date=2020-01-28&end_date=2020-01-28&api_key='+API_KEY
r_test = requests.get(url_test)
json_data_test = r_test.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data_test)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-01-28', 'end_date': '2020-01-28', 'frequency': 'daily', 'data': [['2020-01-28', 118.8, 119.6, 115.2, 118.1, None, 134333.0, 15754191.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)


First I'll begin by setting the url query with set start date and end date for the entire year of 2017, adding my API_KEY to the end of the url string.

I create a Response object (r) using the function **requests.get()**

I set the retrieved Response object to a dict using the **.json()** method.

In [5]:
# 1. Collection of Data from AFX_X for 2017 into a dictionary class
# TEST ODD url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?\
#          &start_date=2017-01-01&end_date=2017-01-05&api_key='+API_KEY
# TEST EVEN url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?\
#           &start_date=2017-01-01&end_date=2017-01-06&api_key='+API_KEY

url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?\
       &start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY

r = requests.get(url)
json_data = r.json()
print(type(r))
print(type(json_data))
#print(json_data)

<class 'requests.models.Response'>
<class 'dict'>


I'm cleaning the dictionary data we received by first iterating through the json_data.items().

The first key is the string 'dataset_data', while its value is a nested dictionary containing the actual API data.

I use a for loop to again, iterate through that dictionary's Data column to go through every entry in the Data column.

I create a new dictionary afx_dict, whose keys are the numerical date as seen in the 'data' columns index 0, and the values are the key:value pairs of the column name and corresponding data values.

In [6]:
# 2. Cleaning of the json_data dictionary file into a more readable dictionary

afx_dict = {}
for key, first_nested_dict in json_data.items(): 
    for index in first_nested_dict['data']: #iterate through the list of lists in Data Column, index = single nested list inside data
        temp_dict = {}
        col=1
        while col < len(index):
            temp_dict[first_nested_dict['column_names'][col]] = index[col]
            col += 1
        afx_dict[index[0]] = temp_dict   #dictionary with date as key, dictionary (temp_dict) of data as value
        
        
#print(afx_dict)
    

In [7]:
# 3. Calculate the highest and lowest opening prices were for the stock in this period

highest = 0.0
highest_date = ''
lowest = 100000000.0
lowest_date = ''
for key, value in afx_dict.items():
    try:
        if value['Open'] > highest:
            highest = value['Open']
            highest_date = key
        if value['Open'] < lowest:
            lowest = value['Open']
            lowest_date = key
    except:
        pass

print("The highest opening price was ", highest, " on date ", highest_date, sep="")
print("The lowest opening price was ", lowest, " on date ", lowest_date, sep="")

    

The highest opening price was 53.11 on date 2017-12-14
The lowest opening price was 34.0 on date 2017-01-24


In [8]:
#for key, value in afx_dict.items():
#    print(key, value['High'], value['Low'], value['Open'], value['Traded Volume'])

In [9]:
# 4. What was the largest change in any one day (based on High and Low price)?

daily_change = 0
daily_change_date = ''
for key, value in afx_dict.items():
    try:
        if value['High'] - value['Low'] > daily_change:
            daily_change = round(value['High'] - value['Low'], 2)
            daily_change_date = key
    except:
        pass
print('The largest daily change (Based on High and Low prices) is', daily_change, 'on date', daily_change_date,'.')

The largest daily change (Based on High and Low prices) is 2.81 on date 2017-05-11 .


In [10]:
# 5. What was the largest change between any two days (based on Closing Price)?
lowest_closing = 100000000
lowest_closing_date = ''
highest_closing = 0
highest_closing_date = ''
for key, value in afx_dict.items():
    try:
        if value['Close'] > highest_closing:
            highest_closing = value['Close']
            highest_closing_date = key
        if value['Close'] < lowest_closing:
            lowest_closing = value['Close']
            lowest_closing_date = key
    except:
        pass
change_in_closing = highest_closing - lowest_closing
print("The largest change between any two day's closing prices are", change_in_closing)
print("The largest change is from", lowest_closing_date, "to", highest_closing_date)

The largest change between any two day's closing prices are 19.03
The largest change is from 2017-01-23 to 2017-12-13


In [11]:
# 6. What was the average daily trading volume during this year?

sum_trading_vol = 0
count = 0
for key, value in afx_dict.items():
    try:
        sum_trading_vol += value['Traded Volume']
        count += 1
    except:
        none_count += 1
        pass
print('The average daily trading volume during this year is', sum_trading_vol/count)

The average daily trading volume during this year is 89124.33725490196


In [12]:
# 7. (Optional) What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)

def median(list):
    if len(list)%2==1:
        median_index = math.ceil(len(list)/2)
        return list[median_index-1]
    else:
        median_index = math.ceil(len(list)/2)
        mean = (list[median_index-1]+list[median_index])/2
        return mean
    

trading_vol_list = []
for key, value in afx_dict.items():
    trading_vol_list.append(value['Traded Volume'])
trading_vol_list.sort()
#print(trading_vol_list)
print()
print("The length of the list is:", len(trading_vol_list))
print("The median of the trading volumes of the data set is:", median(trading_vol_list))
    

    
        




The length of the list is: 255
The median of the trading volumes of the data set is: 76286.0
